In [1]:
import numpy as np
from qiskit.quantum_info.states import Statevector
from qiskit.quantum_info.operators import Pauli, SparsePauliOp

### The `expected_value` function

An MD method does not need to belong to a class (but it can). So there is latitude in how to organize it. For the moment it lives in a file in the top-level of qiskit. This is mostly to avoid cyclic imports. Moving the method definitions to live near the types in their signature would avoid this problem.

In [2]:
from qiskit.operations import expectation_value

To demonstrate `expectation_value`, we create a 2-qubit state and a diagonal operator.

In [3]:
state = Statevector([1, 2, 3, -4])
state = state / np.linalg.norm(state.data)
diagop = Pauli('ZZ')

### API

`expectation_value` is called like this:

```python
expectation_value([oper], state, [qargs])
```
In some cases (e.g. for state of type `Counts`) the first argument may be ommitted and a default value is used. 

`qargs` is optional, and applies an operator to a selected subsystem.

Here we compute the expecation value with the operator and state constructed above.

In [4]:
expectation_value(diagop, state)

0.1333333333333333

We can also sample counts and compute the expectation value of a diagonal operator. Recall that this simulates measuring in the $ZZ$ basis.

In [5]:
state_counts = state.sample_counts(1_000_000)

In [6]:
expectation_value(diagop, state_counts)

0.131966

Because you can only measure a diagonal operator, passing the operator is not required, it is assumed to be diagonal with no `I` operators.

In [7]:
expectation_value(state_counts)

0.131966

We try another diagonal operator `ZI`.

In [8]:
expectation_value(Pauli('ZI'), state)

-0.6666666666666665

Any diagonal operator, including `ZI`, also works with `Counts`.

In [9]:
expectation_value(Pauli('ZI'), state_counts)

-0.666502

By passing `qargs` we can select a subsystem to use with a smaller operator. Note that qubits are counted from right to left, starting with `0`.

In [10]:
[expectation_value(Pauli('Z'), state, [qbit_num]) for qbit_num in (0,1)]

[-0.33333333333333326, -0.6666666666666665]

The same works with `Counts` again. But, passing the operator explicitly is not yet supported. Instead the operator is assumed to be the non-trivial diagonal one-qubit operator on each qubit specified.

In [11]:
[expectation_value(state_counts, [qbit_num]) for qbit_num in (0,1)]

[-0.331216, -0.666502]

For taking the expectation value of `Counts` it would be more performant to allow simply passing `ZZ` rather than `Paul(ZZ)`. So we do allow this. In fact, a `Pauli` is translated to a string internally. A string passed as the first argument is interpreted as Pauli operator.

In [12]:
expectation_value('ZZ', state_counts)

0.131966

In order to make the interface more uniform, we allow this for subclasses of `QuantumState` as well.

In [13]:
[expectation_value('ZZ', state), expectation_value('XY', state)]

[0.1333333333333333, 0.0]

There are arguments for and against allowing the latter signature. For the moment, it is present.

### Diagonal operators and `Counts`

Because phase information is lost when building an empirical probability distribution as `Counts` (or `QuasiDistribution`) only diagonal operators will give sensible results. So:

*We assume that any operator passed to `expected_value` is diagonal*

To see a bit more about how this works, let's use the operator `XZ`. First with the original state.

In [14]:
expectation_value(Pauli('XZ'), state)

0.7333333333333332

And then with the counts

In [15]:
expectation_value(Pauli('XZ'), state_counts)

0.131966

Now `state` and `state_counts` give different results. In fact the latter result is the same that we got with `ZZ`. This holds in general. Since the caller passed `XZ`, we assume that when collecting counts, the qubits were measured in `XZ` basis. Otherwise, we would not, in general, be able to compute the expectation value.

In constrast, the expectation value of any operator may be computed for `state`. So we do just that.

The user needs to know what basis the counts were measured in. But, the algorithm in `expectation_value` does not. So, we allow passing a string with each `0` representing `I` and each `1` representing the measurement-basis operator on the corresponding qubit. In fact, for `Counts` and `QuasiDistribution` this more performant way to pass the operator. For example:

In [16]:
[expectation_value(oper, state_counts) for oper in ('11', '10', '01')]

[0.131966, -0.666502, -0.331216]

#### `SparsePauliOp` and `Counts`

A weighted Pauli sum also works with `Counts` as long as each term is diagonal. Here we use the same operator for the states `state` and `state_counts`.

In [17]:
sp_op = SparsePauliOp(['ZZ', 'ZI'], [1.0, 2.0])
[expectation_value(sp_op, state), expectation_value(sp_op, state_counts)]

[(-1.1999999999999997+0j), (-1.201038+0j)]

#### `QuasiDistribution`

We can take the expection value of `QuasiDistribution`.

In [18]:
from qiskit import QuantumCircuit
from qiskit.primitives import Sampler

bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

# executes a Bell circuit
sampler = Sampler()
result = sampler.run(circuits=[bell]).result()
print([q.binary_probabilities() for q in result.quasi_dists])


[{'00': 0.4999999999999999, '01': 0.0, '10': 0.0, '11': 0.4999999999999999}]


The instance of `SamplerResult` contains `QuasiDistribution`s:

In [19]:
qdist = result.quasi_dists[0]
type(qdist)

qiskit.result.distributions.quasi.QuasiDistribution

The simplest call to `expectation_value` is

In [20]:
expectation_value(qdist)

0.9999999999999998

The semantics of `expectation_value` for `Counts` and `QuasiDistribution` is the same.

In [21]:
[expectation_value(oper, qdist) for oper in ('11', 'ZZ', Pauli('ZZ'), '10', 'ZI', Pauli('ZI'))]

[0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.0, 0.0, 0.0]

`SamplerResult` contains a list of `QuasiDistribution`s (and some metadata). We can operate on this directly with a list of expectation values:

In [22]:
expectation_value(result)

[0.9999999999999998]

### `DensityMatrix` and `StabilizerState`

The semantics of `expectation_value` for these is the same as for `Statevector`. We will not give examples here.